# Sprint07 ensemble

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

aims_data = pd.read_csv("train.csv")
aims_data4 = aims_data.loc[:,['GrLivArea','YearBuilt','SalePrice']]

In [2]:
from sklearn.model_selection import train_test_split
X = aims_data4.iloc[:,0:2].values
y = np.log(aims_data4.iloc[:,2].values)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 1)

# 【問題１】ブレンディングのスクラッチ実装  
## 線形回帰、ＳＶＭ、決定木によるＭＳＥ

In [3]:
##　線形回帰　##
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(X_train,y_train)
y_pred_lin = linear_reg.predict(X_test)
print("線形回帰のＭＳＥ：{}".format(mean_squared_error(y_test, y_pred_lin)))

線形回帰のＭＳＥ：0.045273693264553674


In [4]:
##　SVM ##
from sklearn.svm import SVR
svm_reg = SVR(gamma="auto")
svm_reg.fit(X_train,y_train)
y_pred_svm = svm_reg.predict(X_test)
print("ＳＶＭのＭＳＥ：{}".format(mean_squared_error(y_test, y_pred_svm)))

ＳＶＭのＭＳＥ：0.1590226888735829


In [5]:
## 決定木　##
from sklearn.tree import DecisionTreeRegressor
DTC_reg = DecisionTreeRegressor(random_state=3)
DTC_reg.fit(X_train,y_train)
y_pred_DTC = DTC_reg.predict(X_test)
print("決定木のＭＳＥ：{}".format(mean_squared_error(y_test, y_pred_DTC)))

決定木のＭＳＥ：0.06990153855404337


### ＜１＞決定木と線形回帰の重み付けを変化させた例でＭＳＥが改善（0.8：線形回帰、0.2：ＤＴＣにて改善）

In [6]:
ens_list = np.zeros(10)
for i in range(10):
    ens1 = y_pred_DTC*(1-0.1*i)+y_pred_lin*0.1*i
    ens1_mse = mean_squared_error(y_test,ens1)
    ens_list[i] = ens1_mse
ens_list

array([0.06990154, 0.06386848, 0.05862881, 0.05418254, 0.05052966,
       0.04767018, 0.04560409, 0.0443314 , 0.0438521 , 0.0441662 ])

### ＜２＞　特徴量の標準化と対数化の組み合わせ（あまりＭＳＥが改善しなかった）。

In [12]:
#パターン２　標準化と対数化を組み合わせる。
from sklearn.model_selection import train_test_split
X = aims_data4.iloc[:,0:2].values
y = np.log(aims_data4.iloc[:,2].values)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 1)
# Xの正規化
scaler_X = StandardScaler()
X_train_std = scaler_X.fit_transform(X_train)
X_test_std = scaler_X.transform(X_test)
# Xの対数化
X_train_log = np.log(X_train)
X_test_log = np.log(X_test)

In [23]:
linear_reg.fit(X_train_std,y_train)
y_pred_stdlin = linear_reg.predict(X_test_std)
print("線形回帰の特徴量正規化の場合：{}".format(mean_squared_error(y_test, y_pred_stdlin)))

線形回帰の特徴量正規化の場合：0.0452736932645537


In [24]:
linear_reg.fit(X_train_log,y_train)
y_pred_loglin = linear_reg.predict(X_test_log)
print("線形回帰の特徴量対数化の場合：{}".format(mean_squared_error(y_test, y_pred_loglin)))

線形回帰の特徴量対数化の場合：0.04739709490714278


In [25]:
# 線形回帰の正規化と対数化の組み合わせ(重み付け)
ens_list = np.zeros(10)
for i in range(10):
    ens1 = y_pred_stdlin*(1-0.1*i)+y_pred_loglin*0.1*i
    ens1_mse = mean_squared_error(y_test,ens1)
    ens_list[i] = ens1_mse
ens_list
# アンサンブルしてもあまり改善されない。

array([0.04527369, 0.04514735, 0.04509627, 0.04512045, 0.0452199 ,
       0.04539461, 0.04564458, 0.04596981, 0.04637031, 0.04684607])

In [26]:
DTC_reg.fit(X_train_std,y_train)
y_pred_stdDTC = DTC_reg.predict(X_test_std)
print("ＤＴＣの特徴量正規化の場合：{}".format(mean_squared_error(y_test, y_pred_stdDTC)))

ＤＴＣの特徴量正規化の場合：0.06985811321961863


In [27]:
DTC_reg.fit(X_train_log,y_train)
y_pred_logDTC = DTC_reg.predict(X_test_log)
print("ＤＴＣの特徴量対数化の場合：{}".format(mean_squared_error(y_test, y_pred_logDTC)))

ＤＴＣの特徴量対数化の場合：0.07023579732458154


In [28]:
DTC_reg.fit(X_train,y_train)
y_pred_DTC = DTC_reg.predict(X_test)
print("ＤＴＣのＭＳＥ：{}".format(mean_squared_error(y_test, y_pred_DTC)))

ＤＴＣのＭＳＥ：0.06990153855404337


### ＜３＞決定木の特徴量標準化と特徴量対数化の組み合わせ(重み付け)で、わずかにＭＳＥが改善した。

In [29]:
# 決定木の標準化と対数化の組み合わせ(重み付け)
ens_list = np.zeros(10)
for i in range(10):
    ens1 = y_pred_stdDTC*(1-0.1*i)+y_pred_logDTC*0.1*i
    ens1_mse = mean_squared_error(y_test,ens1)
    ens_list[i] = ens1_mse
ens_list
# 標準化と対数化の半々ぐらいでわずかに改善する。

array([0.06985811, 0.0696555 , 0.06950631, 0.06941054, 0.06936818,
       0.06937924, 0.06944372, 0.06956161, 0.06973292, 0.06995765])

### ＜４＞ランダムフォレストと線形回帰の組み合わせ（重み付け）。ＲＦＣの重み０．３、線形回帰の重み０．７ぐらいで改善する。

In [11]:
#　RFCと線形回帰の組み合わせ（重み付け）
from sklearn.ensemble import RandomForestRegressor
rfc_reg = RandomForestRegressor(n_estimators=100, random_state=7)
rfc_reg.fit(X_train, y_train)
y_pred_rfc = rfc_reg.predict(X_test)
print("評価値：{}".format(mean_squared_error(y_test, y_pred_rfc)))

評価値：0.04922923371678879


In [12]:
linear_reg.fit(X_train,y_train)
y_pred_lin = linear_reg.predict(X_test)
print("評価値：{}".format(mean_squared_error(y_test, y_pred_lin)))

評価値：0.045273693264553674


In [13]:
# RFCと線形回帰の組み合わせ(重み付け)
ens_list = np.zeros(10)
for i in range(10):
    ens1 = y_pred_rfc*(1-0.1*i)+y_pred_lin*0.1*i
    ens1_mse = mean_squared_error(y_test,ens1)
    ens_list[i] = ens1_mse
ens_list
# ＲＦＣの重み0.7、線形回帰の重み0.3ぐらいで改善する。

array([0.04922923, 0.04736676, 0.04583027, 0.04461976, 0.04373524,
       0.04317669, 0.04294413, 0.04303755, 0.04345695, 0.04420233])

## 【問題2】バギングのスクラッチ実装 （→　ＤＴＣの評価値が改善した）

In [35]:
from sklearn.model_selection import train_test_split
X = aims_data4.iloc[:,0:2].values
y = np.log(aims_data4.iloc[:,2].values)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 1)

# samples = len(X_train)
# index = np.random.choice(samples,int(samples*0.8))
# y_train[index].shape

In [40]:
def bagging(X_train,X_test,y_train,y_test,model,iter_num,):
    samples = len(X_train)
    bagging = np.array([])
    for i in range(iter_num):
        index = np.random.choice(samples,int(samples*0.8))
        model.fit(X_train[index,:], y_train[index])
        y_pred = model.predict(X_test)
        if i == 0:
            bagging = np.append(bagging,y_pred)
        else:
            bagging = np.vstack((bagging,y_pred))
            # （1、292）のデータが行方向にスタックされていく。
    bagging = np.mean(bagging, axis = 0)
    return bagging

bagging_pred1 = bagging(X_train,X_test,y_train,y_test,linear_reg,10)
print("linear_reg評価値：{}".format(mean_squared_error(y_test, bagging_pred1)))

bagging_pred2 = bagging(X_train,X_test,y_train,y_test,DTC_reg,10)
print("ＤＴＣ評価値：{}".format(mean_squared_error(y_test, bagging_pred2)))

bagging_pred3 = bagging(X_train,X_test,y_train,y_test,rfc_reg,10)
print("ＲＦＣ評価値：{}".format(mean_squared_error(y_test, bagging_pred3)))

bagging_pred4 = bagging(X_train,X_test,y_train,y_test,svm_reg,10)
print("ＲＦＣ評価値：{}".format(mean_squared_error(y_test, bagging_pred4)))    

linear_reg評価値：0.04554747202492565
ＤＴＣ評価値：0.045733102775805716
ＲＦＣ評価値：0.04633056954627312
ＲＦＣ評価値：0.16352211258993687


## 【問題３】スタッキングのスクラッチ実装

In [14]:
from sklearn.model_selection import train_test_split
X = aims_data4.iloc[:,0:2].values
y = np.log(aims_data4.iloc[:,2].values)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 1)

In [15]:
rfc_reg.fit(X_train,y_train)
y_pred = rfc_reg.predict(X_test)
print("評価値：{}".format(mean_squared_error(y_test, y_pred)))

評価値：0.04922923371678879


In [16]:
linear_reg.fit(X_train,y_train)
y_pred = linear_reg.predict(X_test)
print("評価値：{}".format(mean_squared_error(y_test, y_pred)))

評価値：0.045273693264553674


### ３つのモデルでそれぞれ４回のＣＶによりブレンドデータを作成。  
### ラストモデルは、線形回帰を使用した→評価値が改善した。

In [17]:
models = [DecisionTreeRegressor(), LinearRegression(), RandomForestRegressor(n_estimators=100, random_state=7)]
from sklearn.model_selection import KFold
kf = KFold(n_splits=4, shuffle =True, random_state=15)

### ブレンドデータ（blend_all）、ブレンドテストデータ、ラベルリストの作成 ###
###（ラベルリストは4回のループで同一の値が繰り返し出力される）###
blend_all = np.zeros((len(X_train),len(models)))
blend_test = np.zeros((len(models), len(X_test), 4))
model_count = 0
for i in range(len(models)):
    predict_list = []
    label_list = []
    count = 0
    for t_index,v_index in kf.split(X_train):
        models[i].fit(X_train[t_index],y_train[t_index])
        blend_data = models[i].predict(X_train[v_index])
        test_data = models[i].predict(X_test)
        label_data = y_train[v_index]
        
        predict_list = np.append(predict_list, blend_data)
        blend_test[model_count ,:, count] = test_data
        label_list = np.append(label_list,label_data)

        count += 1
    blend_all[:,i] = predict_list 
    model_count += 1
    
### ブレンドテストデータの、各モデルのmeanを計算。
blend_test = np.mean(blend_test, axis=2).T

### ブレンドデータとラベルデータを最終モデルにフィット。平均化されたブレンドテストデータによる予測値を出力する。
last_model = models[1]
last_model.fit(blend_all, label_list)
last_pred = last_model.predict(blend_test)

mean_squared_error(y_test, last_pred)

0.042148402049494375